In [1]:
import nltk
import sys
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [2]:
messages = pd.read_csv('SPAM.csv',delimiter='\t',names=['label','message'])

In [3]:
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
mess=messages['message'][0]

In [5]:
mess

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [6]:
nonpunc=[char for char in mess if char not in string.punctuation]
nonpunc
# get rid of all comma, keep space still

['G',
 'o',
 ' ',
 'u',
 'n',
 't',
 'i',
 'l',
 ' ',
 'j',
 'u',
 'r',
 'o',
 'n',
 'g',
 ' ',
 'p',
 'o',
 'i',
 'n',
 't',
 ' ',
 'c',
 'r',
 'a',
 'z',
 'y',
 ' ',
 'A',
 'v',
 'a',
 'i',
 'l',
 'a',
 'b',
 'l',
 'e',
 ' ',
 'o',
 'n',
 'l',
 'y',
 ' ',
 'i',
 'n',
 ' ',
 'b',
 'u',
 'g',
 'i',
 's',
 ' ',
 'n',
 ' ',
 'g',
 'r',
 'e',
 'a',
 't',
 ' ',
 'w',
 'o',
 'r',
 'l',
 'd',
 ' ',
 'l',
 'a',
 ' ',
 'e',
 ' ',
 'b',
 'u',
 'f',
 'f',
 'e',
 't',
 ' ',
 'C',
 'i',
 'n',
 'e',
 ' ',
 't',
 'h',
 'e',
 'r',
 'e',
 ' ',
 'g',
 'o',
 't',
 ' ',
 'a',
 'm',
 'o',
 'r',
 'e',
 ' ',
 'w',
 'a',
 't']

In [29]:
nonpunc=''.join(nonpunc)

In [36]:
nonpunc# form into a string

'Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat'

In [7]:
s='Go'
s.lower()

'go'

In [38]:
import nltk
nltk.download('stopwords')
nclean_mess = [word for word in nonpunc.split() if word.lower() not in stopwords.words('english')]

[nltk_data] Downloading package stopwords to C:\Users\ZHAOFENG
[nltk_data]     SHANG\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [39]:
nclean_mess#  after get rid of all stopwords.

['Go',
 'jurong',
 'point',
 'crazy',
 'Available',
 'bugis',
 'n',
 'great',
 'world',
 'la',
 'e',
 'buffet',
 'Cine',
 'got',
 'amore',
 'wat']

In [76]:
def text_process(mess):

    nonpunc = [char for char in mess if char not in string.punctuation]# in a string for every words

    nonpunc = ''.join(nonpunc)
    # first need split the nonpunc('string'),then use lower case word to check stopwords
    return [words for words in nonpunc.split() if words.lower() not in stopwords.words('english')]

In [77]:
bow_CountVectorizer = CountVectorizer(analyzer=text_process)

In [78]:
bow_CountVectorizer.fit_transform((messages['message']))

<5572x11425 sparse matrix of type '<class 'numpy.int64'>'
	with 50548 stored elements in Compressed Sparse Row format>

In [79]:
#a sample
message4 = messages['message'][3]
message4

'U dun say so early hor... U c already then say...'

In [80]:
transformer=TfidfTransformer()

In [81]:
tfidf=transformer.fit_transform(bow_CountVectorizer.fit_transform(messages['message']))

In [82]:
tfidf

<5572x11425 sparse matrix of type '<class 'numpy.float64'>'
	with 50548 stored elements in Compressed Sparse Row format>

In [106]:
from sklearn.naive_bayes import MultinomialNB

spam_dect_model = MultinomialNB().fit(tfidf, messages['label'])

In [107]:
all_predictions = spam_dect_model.predict(tfidf)

In [108]:
all_predictions
# 二者的计算粒度不一样，多项式模型以单词为粒度，伯努利模型以文件为粒度，因此二者的先验概率和类条件概率的计算方法都不同。
# 是伯努利朴素贝叶斯，是有无的0，1所以只能是0，1，所以分母是NO或yes的个数
# 是多项式贝叶斯，则可以将这些词求tfidf值，这些值随频率有大有小，所以分母是总词数而不是个数了

array(['ham', 'ham', 'spam', ..., 'ham', 'ham', 'ham'],
      dtype='<U4')

In [63]:
corpus = [
    'This is the first document.',
     'This is the second second document.',
     'And the third one.',
   'Is this the first document?']


In [68]:
vectorizer = CountVectorizer(analyzer=text_process)
X = vectorizer.fit_transform(corpus)

In [70]:
X.toarray()

array([[1, 1, 0, 0, 0],
       [1, 0, 0, 2, 0],
       [0, 0, 1, 0, 1],
       [1, 1, 0, 0, 0]], dtype=int64)

In [72]:
vectorizer.vocabulary_

{'document': 0, 'first': 1, 'one': 2, 'second': 3, 'third': 4}

In [73]:
transformer=TfidfTransformer()

In [75]:
print(transformer.fit_transform(X))#第一句话的first这个词

  (0, 1)	0.777221162079
  (0, 0)	0.62922751467
  (1, 0)	0.30403549376
  (1, 3)	0.952660704834
  (2, 4)	0.707106781187
  (2, 2)	0.707106781187
  (3, 1)	0.777221162079
  (3, 0)	0.62922751467


In [8]:
X = np.random.randint(5, size=(6, 100))  #6*100 个5以下的数组做multinomial
y = np.array([1, 2, 3, 4, 5, 6])  
from sklearn.naive_bayes import MultinomialNB  
#朴素贝叶斯#多项式分布

In [9]:
X

array([[2, 4, 1, 1, 3, 4, 2, 4, 0, 2, 0, 0, 1, 1, 0, 0, 2, 3, 1, 0, 4, 1,
        4, 1, 0, 2, 2, 4, 4, 2, 1, 2, 3, 0, 1, 0, 0, 1, 0, 4, 1, 2, 4, 2,
        3, 4, 0, 3, 1, 3, 0, 2, 2, 0, 3, 2, 3, 1, 4, 1, 3, 0, 0, 0, 4, 1,
        3, 3, 1, 2, 0, 3, 2, 4, 4, 1, 3, 2, 3, 1, 2, 1, 4, 4, 3, 1, 4, 2,
        1, 0, 4, 1, 0, 4, 0, 3, 1, 4, 1, 1],
       [1, 4, 3, 3, 0, 2, 2, 0, 0, 3, 0, 4, 0, 0, 3, 0, 2, 1, 3, 3, 3, 0,
        4, 2, 3, 4, 0, 3, 2, 0, 4, 0, 2, 1, 0, 2, 1, 0, 4, 4, 4, 4, 4, 3,
        0, 3, 2, 2, 0, 4, 4, 1, 0, 4, 2, 2, 0, 4, 4, 1, 3, 2, 4, 3, 4, 0,
        3, 4, 0, 0, 3, 0, 4, 4, 3, 3, 2, 3, 4, 4, 1, 0, 4, 3, 4, 2, 4, 3,
        3, 0, 3, 3, 0, 3, 4, 3, 1, 1, 0, 2],
       [3, 0, 3, 1, 2, 0, 1, 1, 1, 0, 4, 2, 2, 1, 2, 2, 3, 2, 1, 1, 1, 0,
        2, 2, 1, 3, 2, 0, 1, 3, 2, 4, 0, 4, 3, 2, 3, 0, 1, 3, 4, 0, 1, 0,
        3, 3, 2, 0, 2, 0, 4, 3, 4, 3, 3, 0, 2, 1, 0, 2, 1, 3, 0, 1, 3, 2,
        1, 4, 3, 4, 0, 2, 3, 3, 4, 3, 2, 1, 1, 0, 0, 4, 1, 1, 3, 1, 0, 0,
        0, 0, 2, 1, 0,

In [105]:
a=MultinomialNB().fit(X, y)  
a.predict(X[2:3])

array([3])

In [10]:
X[2:3]

array([[3, 0, 3, 1, 2, 0, 1, 1, 1, 0, 4, 2, 2, 1, 2, 2, 3, 2, 1, 1, 1, 0,
        2, 2, 1, 3, 2, 0, 1, 3, 2, 4, 0, 4, 3, 2, 3, 0, 1, 3, 4, 0, 1, 0,
        3, 3, 2, 0, 2, 0, 4, 3, 4, 3, 3, 0, 2, 1, 0, 2, 1, 3, 0, 1, 3, 2,
        1, 4, 3, 4, 0, 2, 3, 3, 4, 3, 2, 1, 1, 0, 0, 4, 1, 1, 3, 1, 0, 0,
        0, 0, 2, 1, 0, 0, 1, 3, 2, 1, 2, 2]])